# Scrapping

## Scrapping players of Barcelona's matchs with their nations 2024 from transfermarket

In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
}

In [4]:
#Players id
ids = ['74857',
 '709955',
 '938145',
 '196948',
 '466794',
 '158863',
 '112515',
 '182712',
 '636695',
 '937956',
 '683840',
 '636703',
 '85370',
 '709966',
 '466810',
 '646739',
 '398184',
 '943837',
 '938158',
 '283170',
 '938144',
 '480267',
 '962110',
 '666974',
 '636688',
 '411975',
 '937955',
 '66100',
 '646740',
 '326330',
 '53622',
 '970633',
 '411295',
 '724520',
 '937958',
 '462250',
 '38253',
 '707572']

#Club competitions to exclude
clubs_match = ['UEFA Euro qualifying U19',
 'Primera Federación - Grupo I',
 'LaLiga','Premier League','FA Cup',
 'Coupe du Roi',
 'UEFA Europa Conference League',
 'UEFA Champions League',
 'UEFA Youth League',
 'Promoción de ascenso a LaLiga2',
 'Jeux olympiques',
 "Supercoupe de l'UEFA",
 'Supercopa',
 'Community Shield',
 'EFL Cup',
 'UEFA Euro qualifying U21',
 'UEFA Europa League']

In [ ]:
url_barca = 'https://www.transfermarkt.fr/fc-barcelone/kader/verein/131/plus/0/galerie/0?saison_id=2023'
# Send a request to the URL
response_barca = requests.get(url_barca, headers=headers)
nations_macth = []

for id in ids:
    url_barca = f'https://www.transfermarkt.fr/abde-ezzalzouli/leistungsdatendetails/spieler/{id}/plus/1?saison=2023&verein=&liga=&wettbewerb=&pos=&trainer_id='
    # Send a request to the URL
    response_barca = requests.get(url_barca, headers=headers)
    if response_barca.status_code == 200:
        soup = BeautifulSoup(response_barca.content, 'html.parser')

        div_boxes = soup.find_all('div', class_='box')[1:]
        for div_box in div_boxes:
            championnat = div_box.find('a').text.strip()
            if championnat in clubs_match:
                continue

            player_id = id
            data_box = div_box.find('tbody').find_all('tr')
            for tr in data_box:
                td_elements = tr.find_all('td', class_="zentriert")
                if len(td_elements) <= 1:
                    continue
                date = td_elements[1].text.strip()

                score = td_elements[4].text.strip()

                teams = tr.find_all('td', class_="no-border-links")
                team_home = teams[0].find('a').text.strip()
                team_away = teams[1].find('a').text.strip()


                nations_macth.append({
                    'Match_date': date,
                    'ID': player_id,
                    'Home_team': team_home,
                    'Away_team': team_away,
                    'Result': score,
                    'Championnat': championnat
                })

In [15]:
df_nations_match = pd.DataFrame(nations_macth)

In [16]:
df_nations_match

,Match_date,ID,Home_team,Away_team,Result,Championnat
0,14 juin 2024,74857,Allemagne,Écosse,5:1,Championnat d'Europe 2024
1,19 juin 2024,74857,Allemagne,Hongrie,2:0,Championnat d'Europe 2024
2,23 juin 2024,74857,Suisse,Allemagne,1:1,Championnat d'Europe 2024
3,29 juin 2024,74857,Allemagne,Danemark,2:0,Championnat d'Europe 2024
4,5 juil. 2024,74857,Espagne,Allemagne,2:1 ap,Championnat d'Europe 2024
...,...,...,...,...,...,...
196,25 juin 2024,38253,France,Pologne,1:1,Championnat d'Europe 2024
197,21 mars 2024,38253,Pologne,Estonie,5:1,EM-Qualifikation Play-Offs
198,26 mars 2024,38253,Pays de Galles,Pologne,4:5 tab,EM-Qualifikation Play-Offs
199,7 juin 2024,38253,Pologne,Ukraine,3:1,Amicaux internationaux


### Transforming date format from dd month YYYY to dd/mm/YYYY


In [17]:
mois_map = {
    "janv.": "01", "févr.": "02", "mars": "03", "avr.": "04", "mai": "05", "juin": "06",
    "juil.": "07", "août": "08", "sept.": "09", "oct.": "10", "nov.": "11", "déc.": "12"
}
def reformatter_date(date):
    for mois, num in mois_map.items():
        if mois in date:
            date = date.replace(mois, num)
            break
    # Ajoute des zéros si le jour ou le mois est à un chiffre
    match = re.match(r"(\d{1,2}) (\d{2}) (\d{4})", date)
    if match:
        jour = match.group(1).zfill(2)  # Remplir à deux chiffres
        mois = match.group(2).zfill(2)
        annee = match.group(3)
        return f"{jour}/{mois}/{annee}"
    return date

In [18]:
df_nations_match['Match_date'] = df_nations_match['Match_date'].apply(reformatter_date)
df_nations_match.head()

,Match_date,ID,Home_team,Away_team,Result,Championnat
0,14/06/2024,74857,Allemagne,Écosse,5:1,Championnat d'Europe 2024
1,19/06/2024,74857,Allemagne,Hongrie,2:0,Championnat d'Europe 2024
2,23/06/2024,74857,Suisse,Allemagne,1:1,Championnat d'Europe 2024
3,29/06/2024,74857,Allemagne,Danemark,2:0,Championnat d'Europe 2024
4,05/07/2024,74857,Espagne,Allemagne,2:1 ap,Championnat d'Europe 2024


### Cleaning score (2:1 ap) --> (2:1)




In [19]:
df_nations_match['Result'] = df_nations_match['Result'].str.split().str[0]
df_nations_match.head()

,Match_date,ID,Home_team,Away_team,Result,Championnat
0,14/06/2024,74857,Allemagne,Écosse,5:1,Championnat d'Europe 2024
1,19/06/2024,74857,Allemagne,Hongrie,2:0,Championnat d'Europe 2024
2,23/06/2024,74857,Suisse,Allemagne,1:1,Championnat d'Europe 2024
3,29/06/2024,74857,Allemagne,Danemark,2:0,Championnat d'Europe 2024
4,05/07/2024,74857,Espagne,Allemagne,2:1,Championnat d'Europe 2024


### Spliting score

In [20]:
df_nations_match[['Home_team_goals', 'Away_team_goals']] = df_nations_match['Result'].str.split(':', expand=True)

# Convertir les nouvelles colonnes en entier
df_nations_match['Home_team_goals'] = df_nations_match['Home_team_goals'].astype(int)
df_nations_match['Away_team_goals'] = df_nations_match['Away_team_goals'].astype(int)

df_nations_match.head()

,Match_date,ID,Home_team,Away_team,Result,Championnat,Home_team_goals,Away_team_goals
0,14/06/2024,74857,Allemagne,Écosse,5:1,Championnat d'Europe 2024,5,1
1,19/06/2024,74857,Allemagne,Hongrie,2:0,Championnat d'Europe 2024,2,0
2,23/06/2024,74857,Suisse,Allemagne,1:1,Championnat d'Europe 2024,1,1
3,29/06/2024,74857,Allemagne,Danemark,2:0,Championnat d'Europe 2024,2,0
4,05/07/2024,74857,Espagne,Allemagne,2:1,Championnat d'Europe 2024,2,1


### reorgnising columns

In [21]:
new_order = [
    "Match_date", "ID", "Home_team", "Away_team", "Home_team_goals", "Away_team_goals", "Championnat"
]

df_nations_match = df_nations_match[new_order]
df_nations_match.head()

,Match_date,ID,Home_team,Away_team,Home_team_goals,Away_team_goals,Championnat
0,14/06/2024,74857,Allemagne,Écosse,5,1,Championnat d'Europe 2024
1,19/06/2024,74857,Allemagne,Hongrie,2,0,Championnat d'Europe 2024
2,23/06/2024,74857,Suisse,Allemagne,1,1,Championnat d'Europe 2024
3,29/06/2024,74857,Allemagne,Danemark,2,0,Championnat d'Europe 2024
4,05/07/2024,74857,Espagne,Allemagne,2,1,Championnat d'Europe 2024


In [22]:
df_nations_match.to_csv('nations_match.csv', index=False)